In [1]:
import tensorflow

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
import keras

Using TensorFlow backend.


In [3]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [5]:
#tbCallBack = keras.callbacks.TensorBoard(log_dir='/data/tb-logs', histogram_freq=0, write_graph=True, write_images=True)


In [6]:
img_width, img_height = 256, 256
train_data_dir = "/data/train"
validation_data_dir = "/data/train"
nb_train_samples = 602813/10
nb_validation_samples = 66491/20 
batch_size = 16
epochs = 50

In [7]:
nr_categories = 997
batch_size = 16
epochs = 50

In [10]:
#model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
from keras.models import load_model
model = load_model("/data/vgg16_1.h5")

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [12]:
# # Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
# for layer in model.layers[:]:
#     print(layer.name)
#     layer.trainable = False

In [13]:
# #Adding custom Layers 
# x = model.output
# x = Flatten()(x)
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(1024, activation="relu")(x)
# predictions = Dense(nr_categories, activation="softmax")(x)

In [14]:
#model_final = Model(input = model.input, output = predictions)
model_final = model

In [15]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [16]:
# creating the final model 
#model_final = Model(input = model.input, output = predictions)

# compile the model 
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])


In [17]:
train__val_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.1)

In [18]:
#validation_datagen = ImageDataGenerator(rescale = 1./255)

In [19]:
train_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

Found 602813 images belonging to 997 classes.


In [20]:
validation_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')

Found 66491 images belonging to 997 classes.


In [21]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [22]:
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=2, write_graph=True, write_images=True)

In [23]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
3768/3767 [==============================] - 846s 224ms/step - loss: 3.8184 - acc: 0.2467 - val_loss: 3.6703 - val_acc: 0.2767

Epoch 00001: val_acc improved from -inf to 0.27674, saving model to vgg16_1.h5


ValueError: If printing histograms, validation_data must be provided, and cannot be a generator.

In [ ]:
!ls